<a href="https://colab.research.google.com/github/davidclizbe/datascience/blob/master/Clizbe_Deep_Learning_Day_4_Afternoon_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [ ]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [ ]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [ ]:
# Answer below:
yelp['text_norm'] = yelp['text'].apply(normalize)
yelp.head()

,text,sentiment,text_norm
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [ ]:
# Answer below:
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = len(set(yelp['text_norm'].apply(lambda x: x.strip().split(' ')).sum()))
yelp['encoded'] = yelp['text_norm'].apply(one_hot, args=[vocab_size])

In [ ]:
yelp.head()

,text,sentiment,text_norm,encoded
0,Wow... Loved this place.,1,wow love place,"[333, 1327, 70]"
1,Crust is not good.,0,crust not good,"[832, 779, 405]"
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti,"[779, 349, 338, 1428]"
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...,"[923, 1588, 211, 561, 616, 126, 238, 882, 1327..."
4,The selection on the menu was great and so wer...,1,select menu great price,"[851, 1019, 962, 931]"


Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [ ]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences

ind_vars = pad_sequences(yelp['encoded'])
ind_vars

array([[   0,    0,    0, ...,  333, 1327,   70],
       [   0,    0,    0, ...,  832,  779,  405],
       [   0,    0,    0, ...,  349,  338, 1428],
       ...,
       [   0,    0,    0, ...,  779, 1005,  660],
       [   0,    0,    0, ...,  953, 1141,  359],
       [   0,    0,    0, ...,  642, 1235,  551]], dtype=int32)

In [ ]:
ind_vars.shape

(976, 83)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [ ]:
# Answer below:
from sklearn.model_selection import train_test_split

target = yelp['sentiment']
X_train, X_test, y_train, y_test = train_test_split(ind_vars, target, test_size=0.2, random_state=21)

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [ ]:
X_train.shape

(780, 83)

In [ ]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

max_words = np.max(X_train)+1
max_len = X_train.shape[1]

model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(SimpleRNN(32, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 83, 100)           162900    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 167,429
Trainable params: 167,429
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [ ]:
# Answer below:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 50
batch_size = 128

model.fit(X_train, y_train, validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs)

Epoch 1/50
7/7 [==============================] - 0s 65ms/step - loss: 0.6934 - accuracy: 0.5026 - val_loss: 0.6929 - val_accuracy: 0.5408
Epoch 2/50
7/7 [==============================] - 0s 32ms/step - loss: 0.6851 - accuracy: 0.7038 - val_loss: 0.6917 - val_accuracy: 0.5255
Epoch 3/50
7/7 [==============================] - 0s 30ms/step - loss: 0.6773 - accuracy: 0.7500 - val_loss: 0.6892 - val_accuracy: 0.5357
Epoch 4/50
7/7 [==============================] - 0s 31ms/step - loss: 0.6664 - accuracy: 0.8000 - val_loss: 0.6866 - val_accuracy: 0.5714
Epoch 5/50
7/7 [==============================] - 0s 34ms/step - loss: 0.6523 - accuracy: 0.8577 - val_loss: 0.6830 - val_accuracy: 0.5765
Epoch 6/50
7/7 [==============================] - 0s 32ms/step - loss: 0.6323 - accuracy: 0.9090 - val_loss: 0.6784 - val_accuracy: 0.5867
Epoch 7/50
7/7 [==============================] - 0s 33ms/step - loss: 0.6023 - accuracy: 0.9256 - val_loss: 0.6685 - val_accuracy: 0.5969
Epoch 8/50
7/7 [===========

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTETomek

In [ ]:
sm = SMOTE()

X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
Counter(y_train_res)

Counter({0: 398, 1: 398})